# STEDI Step Test — Curated Silver Dataset

In this lab, I combine raw STEDI sensor messages with rapid step test data to create a clean, labeled Silver dataset. Sensor readings are aligned with known step test time windows using timestamps and labeled as either `step` or `no_step`. This curated dataset will be used in later machine-learning work.


## Bronze Data Sources

### device_messages_raw
This table contains continuous ultrasonic sensor readings sent from STEDI devices every few milliseconds. Each record includes a timestamp, device ID, sensor type, and a raw distance measurement stored as a string.

### rapid_step_tests_raw
This table contains rapid step test sessions performed by users. Each record defines a start and stop time for a test window, along with metadata identifying the device and test session.


In [0]:
%sql
CREATE OR REPLACE TEMP VIEW device_messages_clean AS
SELECT
  date            AS event_time,
  device_id,
  sensor_type,
  CAST(REGEXP_REPLACE(distance, 'cm', '') AS INT) AS distance_cm,
  'device'        AS source
FROM workspace.bronze.device_messages_raw;


In [0]:
%sql
CREATE OR REPLACE TEMP VIEW step_tests_clean AS
SELECT
  device_id,
  start_time,
  stop_time
FROM workspace.bronze.rapid_step_tests_raw;


In [0]:
%sql
CREATE OR REPLACE TEMP VIEW labeled_sensor_data AS
SELECT
  d.event_time,
  d.device_id,
  d.sensor_type,
  d.distance_cm,
  d.source,
  s.start_time,
  s.stop_time,
  CASE
    WHEN d.event_time BETWEEN s.start_time AND s.stop_time
      THEN 'step'
    ELSE 'no_step'
  END AS step_label
FROM device_messages_clean d
LEFT JOIN step_tests_clean s
  ON d.device_id = s.device_id;


In [0]:
%sql
CREATE TABLE IF NOT EXISTS workspace.silver.curated_step_sensor_data
USING DELTA
AS
SELECT *
FROM labeled_sensor_data;

In [0]:
%sql
SELECT COUNT(*) AS row_count
FROM labeled_sensor_data;


In [0]:
%sql
CREATE OR REPLACE TABLE workspace.silver.curated_step_sensor_data
USING DELTA
AS
SELECT *
FROM labeled_sensor_data;

In [0]:
%sql
SELECT COUNT(*) AS row_count
FROM workspace.silver.curated_step_sensor_data;


In [0]:
%sql
SELECT step_label, COUNT(*) AS count
FROM workspace.silver.curated_step_sensor_data
GROUP BY step_label;


In [0]:
%sql
SELECT *
FROM workspace.silver.curated_step_sensor_data
LIMIT 20;
